# 3. Create Required Azure Services
You should run first jupyter notebook [02_setup_environment.ipynb](./02_setup_environment.ipynb) in this sample to set the global variables. Once it is run, you dont need to repeat it again and again as the variables etc. stored in an environment file. You may need to run it again only in case you delete the ".env" file or your Azure login session expires.

## 3.1. Get global variables
We will read the previously stored variables

In [ ]:
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

azureSubsctiptionId = get_key(envPath, "azureSubsctiptionId")
resourceLocation = get_key(envPath, "resourceLocation")
resourceGroupName = get_key(envPath, "resourceGroupName")
acrServiceName = get_key(envPath, "acrServiceName")
iotHubServiceName = get_key(envPath, "iotHubServiceName")
iotDeviceId = get_key(envPath, "iotDeviceId")
mediaServiceName = get_key(envPath, "mediaServiceName")
storageServiceName = get_key(envPath, "storageServiceName")

## 3.2. Create Resource Group

In [ ]:
!az group create --name $resourceGroupName --location  $resourceLocation

## 3.3. Create Azure Container Registry Service

In [ ]:
!az acr create --name $acrServiceName --resource-group $resourceGroupName --location $resourceLocation --sku Basic --admin-enabled true

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  
Even the execution of above cell seems to be finished, it may take some time (several seconds) to have the changes reflected on Azure Datacenter. So if you immediately run the below cells after the execution of above one, it may give error by complaining that the ACR service doesnt exists...


### 3.3.1. Get access credentials to the ACR service

In [ ]:
%%bash --out output -s "$acrServiceName"
az acr credential show -n $1

In [ ]:
import json
jsonRegDetails = json.loads(output)

acrUserName = jsonRegDetails['username']
acrPassword = jsonRegDetails['passwords'][0]['value']

Now save these information into the **.env** file for further use.

In [ ]:
set_key(envPath, "acrUserName", acrUserName)
set_key(envPath, "acrPassword", acrPassword)
set_key(envPath, "acrServiceFullName", acrServiceName+".azurecr.io")

## 3.4. Create Azure IoT Hub
Below cell may give following error: "IotHub name '<NAME\>' is not available" in case service already exist.


In [ ]:
!az iot hub create --name $iotHubServiceName --resource-group $resourceGroupName --location $resourceLocation --sku S1

In [ ]:
%%bash --out iotHubConnString -s "$resourceGroupName" "$iotHubServiceName"

az iot hub show-connection-string --hub-name $2 --output tsv

In [ ]:
set_key(envPath, "iotHubConnString", iotHubConnString.rstrip())

## 3.5. Create IoT Edge Device
Create an IoT Edge device under the Azure IoT Hub service created in above cells. In the further steps, when we connect / match the physical device with this cloud based one through the device connection string, everything in this cloud one will be cloned into the physical device (all IoT Edge modules etc.)

In this step we are also printing out and saving the IoT Edge device connection string which will be used in !!!further steps!!!.

In [ ]:
# Install iot hub CLI extensions
!az extension add --name azure-cli-iot-ext

In [ ]:
%%bash --out iotEdgeDeviceConnString -s "$resourceGroupName" "$iotHubServiceName" "$iotDeviceId"
# Create edge device if not already exists
az iot hub device-identity show --resource-group $1 --hub-name $2 --device-id $3 &> /dev/null

if [ $? -ne 0 ]; then
    az iot hub device-identity create --hub-name $2 --device-id $3 --edge-enabled &> /dev/null
fi

az iot hub device-identity show-connection-string --hub-name $2 --device-id $3 --output tsv

In [ ]:
set_key(envPath, "iotEdgeDeviceConnString", iotEdgeDeviceConnString.rstrip())

In [ ]:
# Print out IoT Edge Device deteails and store its connection string...
print('Azure IoT Hub Name: {}'.format(iotHubServiceName))
print('Azure IoT device id : {}'.format(iotDeviceId))
print('IoT Edge Device Connection String: {}'.format(iotEdgeDeviceConnString))
print('IoT Hub Connection String: {}'.format(iotHubConnString))

## 3.6. Create Azure Storage Services
Required by Azure Media Services

In [ ]:
!az storage account create --name $storageServiceName --resource-group $resourceGroupName --location $resourceLocation --sku Standard_LRS

## 3.7. Create Azure Media Services

Create media services for archiving edge streams

In [ ]:
!az ams account create --name $mediaServiceName --resource-group $resourceGroupName --storage-account $storageServiceName --location $resourceLocation

## 3.8. Create Azure Service Principal


In [ ]:
%%bash --out output -s "$mediaServiceName" "$resourceGroupName" 

az ams account sp create --account-name $1 --resource-group $2 --name lvasp$2 --output json

if [ $? -ne 0 ]; then
    az ams account sp reset-credentials --account-name $1 --resource-group lvasp$2 --name $2 --output json
fi

In [ ]:
import json

amssp = json.loads(output)

aadTenantId = amssp['AadTenantId']
aadServicePrincipalId = amssp['AadClientId']
aadServicePrincipalSecret = amssp['AadSecret']

set_key(envPath, "aadTenantId", aadTenantId)
set_key(envPath, "aadServicePrincipalId", aadServicePrincipalId)
set_key(envPath, "aadServicePrincipalSecret", aadServicePrincipalSecret)